## Предсказание неэкспрессирующихся и высоко-экспрессирующихся генов.
Предсказание использует: 1500bp промотора, 1500bp терминатора или оба
Неэкспрессирующиеся гены помеченны как 1
Экспрессирующиеся гены помеченны как 0.
Для того, чтобы запустить обучение нейронной сети, понадобится:
python 2
tensorflow 1.10.1
keras 2.2.4

In [ ]:
import pandas as pd
import Bio.SeqIO
import numpy as np
import math
import random
import pickle
import os
from sys import argv
from keras.metrics import mean_squared_error
from keras.layers import Dense,Activation,Flatten,Dropout,Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.models import Sequential,Model
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from itertools import izip
from datetime import datetime
from keras import backend as K
from sklearn.metrics import confusion_matrix

Пути до необходимых библиотек

Данные файлы с последовательностями промоторов и терминаторов генов храняться по ссылке:
    https://de.cyverse.org/dl/d/0F63CD73-C1BE-4588-8FEF-E9EF57FCA117/data.tar.gz

In [ ]:
SEQ_FILE_PRO='../../../data/pseudogene_model_data/promoters.fa'
SEQ_FILE_PRO_DSHUFFLE='../../../data/pseudogene_model_data/promoters.dishuffle.fa'
SEQ_FILE_TER='../../../data/pseudogene_model_data/terminators.fa'
SEQ_FILE_TER_DSHUFFLE='../../../data/pseudogene_model_data/terminators.dishuffle.fa'
DATA_FILE='../../../data/pseudogene_model_data/un_vs_high.csv'
RESULT_DIRECTORY='../../../data/pseudogene_model_data/trained_models_un_high_expr/'

if not os.path.exists(RESULT_DIRECTORY):
   os.makedirs(RESULT_DIRECTORY)

data=pd.read_csv(DATA_FILE,sep='\t')
data=data.set_index('Geneid')

Пример входных данных:

In [14]:
#import pandas as pd
gene_df=pd.read_csv("un_vs_high.csv", sep="\t")
gene_df.head(5)

,Geneid,max_TPM,category,family_id,subsample_assignment1,subsample_assignment2,subsample_assignment3,subsample_assignment4,subsample_assignment5,subsample_assignment6,subsample_assignment7,subsample_assignment8,subsample_assignment9,subsample_assignment10
0,GRMZM2G002616,383.476196,highly-expressed,0,subsample2,subsample2,subsample3,subsample4,subsample4,subsample2,subsample3,subsample1,subsample3,subsample5
1,GRMZM2G112336,389.840179,highly-expressed,0,subsample2,subsample2,subsample3,subsample4,subsample4,subsample2,subsample3,subsample1,subsample3,subsample5
2,GRMZM2G150404,399.394257,highly-expressed,0,subsample2,subsample2,subsample3,subsample4,subsample4,subsample2,subsample3,subsample1,subsample3,subsample5
3,GRMZM2G142898,407.575531,highly-expressed,0,subsample2,subsample2,subsample3,subsample4,subsample4,subsample2,subsample3,subsample1,subsample3,subsample5
4,GRMZM2G022019,408.910858,highly-expressed,0,subsample2,subsample2,subsample3,subsample4,subsample4,subsample2,subsample3,subsample1,subsample3,subsample5


Создание пустых списков

In [11]:
geneIDs=[]
categories=[]

seqs=[]
seqs_pro=[]
seqs_ter=[]

seqs_dshuffle=[]
seqs_pro_dshuffle=[]
seqs_ter_dshuffle=[]

seqs_sshuffle=[]
seqs_pro_sshuffle=[]
seqs_ter_sshuffle=[]


Прочтение последовательностей и генерирование перемешанных последовательностей. Перемешивание последовательностей
происходит двумя типами: однонуклеотидное и двунуклеотидное. Это действие производиться для того, чтобы убрать
биологических смысл последовательности и посмотреть, что CNN найдет в данных, лищенных смысла. 

In [ ]:
for x,y in izip(Bio.SeqIO.parse(SEQ_FILE_PRO,'fasta'), Bio.SeqIO.parse(SEQ_FILE_TER,'fasta') ):
     geneID=x.id

     seq_pro=x.seq.tostring()
     seq_pro_sshuffle=list(seq_pro)
     random.shuffle(seq_pro_sshuffle)
     seq_pro_sshuffle=''.join(seq_pro_sshuffle)

     seq_ter=y.seq.tostring()
     seq_ter_sshuffle=list(seq_ter)
     random.shuffle(seq_ter_sshuffle)
     seq_ter_sshuffle=''.join(seq_ter_sshuffle)

     seq=seq_pro+seq_ter
     seq_sshuffle=seq_pro_sshuffle+seq_ter_sshuffle

     if geneID in data.index:
         seqs.append(seq)
         seqs_sshuffle.append(seq_sshuffle)
         seqs_pro.append(seq_pro)
         seqs_pro_sshuffle.append(seq_pro_sshuffle)
         seqs_ter.append(seq_ter)
         seqs_ter_sshuffle.append(seq_ter_sshuffle)
         geneIDs.append(geneID)
         category=data['category'][geneID]
         if category=='expressed':
              categories.append(0)
         else:
              categories.append(1)

for x,y in izip(Bio.SeqIO.parse(SEQ_FILE_PRO_DSHUFFLE,'fasta'), Bio.SeqIO.parse(SEQ_FILE_TER_DSHUFFLE,'fasta') ):
     geneID=x.id
     seq_pro_dshuffle=x.seq.tostring()
     seq_ter_dshuffle=y.seq.tostring()
     seq_dshuffle=seq_pro_dshuffle+seq_ter_dshuffle
     if geneID in data.index:
         seqs_dshuffle.append(seq_dshuffle)
         seqs_pro_dshuffle.append(seq_pro_dshuffle)
         seqs_ter_dshuffle.append(seq_ter_dshuffle)


На данном этапе каждая последовательность кодируется матрицей размером 4Хдлина последовательности.
Чтобы исключить влияние таких очевидных вещей как старт или стоп кодон они маскируются нулями. 
Нейронная сеть не должна опираться на эти очевидные признаки, тк во многих данных может быть потеряны старты/стопы
транскрипции.

In [ ]:
dict={'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],'T':[0,0,0,1]}

def one_hot_encoding(seq):
    one_hot_encoded=np.zeros(shape=(4,len(seq)))
    for i,nt in enumerate(seq):
        one_hot_encoded[:,i]=dict[nt]
    return one_hot_encoded

one_hot_seqs=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs],dtype=np.float32),3)
one_hot_seqs_pro=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_pro],dtype=np.float32),3)
one_hot_seqs_ter=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_ter],dtype=np.float32),3)


one_hot_seqs[:,:,1000:1003,:]=0
one_hot_seqs[:,:,1997:2000,:]=0
one_hot_seqs_pro[:,:,1000:1003,:]=0
one_hot_seqs_ter[:,:,497:500,:]=0

one_hot_seqs_dshuffle=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_dshuffle],dtype=np.float32),3)
one_hot_seqs_pro_dshuffle=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_pro_dshuffle],dtype=np.float32),3)
one_hot_seqs_ter_dshuffle=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_ter_dshuffle],dtype=np.float32),3)

one_hot_seqs_sshuffle=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_sshuffle],dtype=np.float32),3)
one_hot_seqs_pro_sshuffle=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_pro_sshuffle],dtype=np.float32),3)
one_hot_seqs_ter_sshuffle=np.expand_dims(np.array([ one_hot_encoding(seq) for seq in seqs_ter_sshuffle],dtype=np.float32),3)

geneIDs=np.array(geneIDs)
categories=np_utils.to_categorical(np.array(categories),2)



Архитектура нейронной сети: 
Три группы из двух сверточных слоев, 
одного пуллингового и одного dropout-слоя. Они формируют вектор признаков 
для полносвязных слоев стандартной нейронной сети (представленной в самом конце). 
Далее нейронная сеть состоит из двух полносвязных 
слоев с dropout и из решающего слоя. Везде активационные функции - это ReLU, кроме решающего слоя, 
где использована softmax функция.
Пуллинг и дропаут слои позволяют, в том числе, не переобучать слои.

In [14]:
def build(window_size):
                model=Sequential()

                model.add(Conv2D(64,kernel_size=(4,8),padding='valid',input_shape=[4,window_size,1]))
                model.add(Activation('relu'))
                model.add(Conv2D(64,kernel_size=(1,8),padding='same'))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(1,8),strides=(1,8),padding='same'))
                model.add(Dropout(0.25))

                model.add(Conv2D(128,kernel_size=(1,8),padding='same'))
                model.add(Activation('relu'))
                model.add(Conv2D(128,kernel_size=(1,8),padding='same'))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(1,8),strides=(1,8),padding='same'))
                model.add(Dropout(0.25))

                model.add(Conv2D(64,kernel_size=(1,8),padding='same'))
                model.add(Activation('relu'))
                model.add(Conv2D(64,kernel_size=(1,8),padding='same'))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(1,8),strides=(1,8),padding='same'))
                model.add(Dropout(0.25))

                model.add(Flatten())
                model.add(Dense(128))
                model.add(Activation('relu'))
                model.add(Dropout(0.25))
                model.add(Dense(64))
                model.add(Activation('relu'))
                model.add(Dense(2))
                model.add(Activation('softmax'))
                return model


Обучение модели
Распределение на обучающую и тестовую выборку
EarlyStopping строка позволяет остановить обучение если потеря данных больше некоторого порога. Поэтому нейросеть
может останавливаться и не доходить до конца по мере обучения.


In [15]:
def train_models(TRAINING_ONE_HOT_SEQS,TESTING_ONE_HOT_SEQS,training_indices,testing_indices,split,test_subsample,shuffle,predictor):
      
      TRAINING_ONE_HOT_SEQS=TRAINING_ONE_HOT_SEQS[training_indices]
      TESTING_ONE_HOT_SEQS=TESTING_ONE_HOT_SEQS[testing_indices]
      TRAINING_CATEGORIES=categories[training_indices]
      TESTING_CATEGORIES=categories[testing_indices]
      TESTING_GENES=geneIDs[testing_indices]

      LENGTH=TRAINING_ONE_HOT_SEQS.shape[2]
      
      callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0)]
      model=build(LENGTH)
      model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
      model.fit(x=TRAINING_ONE_HOT_SEQS,y=TRAINING_CATEGORIES,batch_size=256,epochs=40,validation_data=(TESTING_ONE_HOT_SEQS,TESTING_CATEGORIES),callbacks=callbacks)

      prediction=model.predict(TESTING_ONE_HOT_SEQS)
      predicted_categories= np.argmax(prediction,axis=1)
      real_categories=np.argmax(TESTING_CATEGORIES,axis=1)
      tn,fp,fn,tp=confusion_matrix(real_categories,predicted_categories).ravel()
      test_total=tp+tn+fp+fn
      accuracy=(tp+tn)/(test_total*1.0)

      now=datetime.now().strftime('%Y%m%d%H%M%S')
      model.save(RESULT_DIRECTORY+"MODEL_"+now)
      pickle.dump([TESTING_GENES,TESTING_CATEGORIES,prediction],open(RESULT_DIRECTORY+"PICKLE_"+now,'wb'))
      with open(RESULT_DIRECTORY+'SUMMARY_FILE', "a") as result_file:
         result_file.write("\t".join([now,split,test_subsample,shuffle,str(tp),str(tn),str(fp),str(fn),str(accuracy),str(test_total),predictor]))
         result_file.write("\n")
      del model
      K.clear_session()


Для статистически достоверной оценки работы модели она запускается 10 раз (каждый раз) с 5-фолд кросс-валидацией. 
(При разбиении выборки учитываются семейства).
Три типа данных: PROMOTER, TERMINATOR, PROMOTER+TERMINATOR.
Во время запуска обеспечивается динуклеотидное и мононуклеотидное перемешивание данных обучения.
Всего тренируется 10\*5\*3\*3=450 моделей.
В тестовой и тренировочной выборках нет необходимости производит балансировку, тк количество высоко-экспрессируемых и неэкспрессируемых генов совпадают.

In [ ]:
splits=data.columns.values[-10:]
subsamples=['subsample1','subsample2','subsample3','subsample4','subsample5']

for split in splits:
   for test_subsample in subsamples:
       testing_genes = data[data[split]==test_subsample].index
       testing_genes = np.array(list(set(testing_genes).intersection(set(geneIDs))))
       training_genes= data[data[split]!=test_subsample].index
       training_genes= np.array(list(set(training_genes).intersection(set(geneIDs))))
       training_indices=np.array([np.where(geneIDs==element)[0][0] for element in training_genes])
       testing_indices=np.array([np.where(geneIDs==element)[0][0] for element in testing_genes])
       np.random.shuffle(training_indices)
       train_models(one_hot_seqs,               one_hot_seqs,      training_indices,testing_indices,split,test_subsample,'None','Pro_and_Ter')
       train_models(one_hot_seqs_pro,           one_hot_seqs_pro,  training_indices,testing_indices,split,test_subsample,'None','Pro')
       train_models(one_hot_seqs_ter,           one_hot_seqs_ter,  training_indices,testing_indices,split,test_subsample,'None','Ter')
       train_models(one_hot_seqs_dshuffle,      one_hot_seqs,      training_indices,testing_indices,split,test_subsample,'D_Shuffle','Pro_and_Ter')
       train_models(one_hot_seqs_pro_dshuffle,  one_hot_seqs_pro,  training_indices,testing_indices,split,test_subsample,'D_Shuffle','Pro')
       train_models(one_hot_seqs_ter_dshuffle,  one_hot_seqs_ter,  training_indices,testing_indices,split,test_subsample,'D_Shuffle','Ter')
       train_models(one_hot_seqs_sshuffle,      one_hot_seqs,      training_indices,testing_indices,split,test_subsample,'S_Shuffle','Pro_and_Ter')
       train_models(one_hot_seqs_pro_sshuffle,  one_hot_seqs_pro,  training_indices,testing_indices,split,test_subsample,'S_Shuffle','Pro')
       train_models(one_hot_seqs_ter_sshuffle,  one_hot_seqs_ter,  training_indices,testing_indices,split,test_subsample,'S_Shuffle','Ter')



На выходе после обучения нейронной сети:

In [18]:
#import pandas as pd
gene_df=pd.read_csv("output_un_vs_high .csv", sep=",")
gene_df.head(5)

,20190519230105,subsample_assignment1,subsample1,None,651,665,193,42,0.8484848484848485,1551,Pro_and_Ter
0,20190519231618,subsample_assignment1,subsample1,None,558,721,137,135,0.824629,1551,Pro
1,20190519232612,subsample_assignment1,subsample1,None,566,671,187,127,0.797550,1551,Ter
2,20190519233230,subsample_assignment1,subsample1,D_Shuffle,693,0,858,0,0.446809,1551,Pro_and_Ter
3,20190519233924,subsample_assignment1,subsample1,D_Shuffle,693,0,858,0,0.446809,1551,Pro
4,20190519234349,subsample_assignment1,subsample1,D_Shuffle,654,25,833,39,0.437782,1551,Ter
